# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = X[:6709,:], X[6709:8386,:], X[8386:,:], Y[:6709], Y[6709:8386], Y[8386:]


In [ ]:
!pip install regex

In [2]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
# write everything in a pipeline

#import logisticregression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

model = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression())
])
model.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('log_reg', LogisticRegression())])

In [3]:

# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval

Y_pred=model.predict(X_test)

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)
#Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
rp=classification_report_interval(y_true=Y_test,y_pred=Y_pred, labels=[0,1],alpha=0.05)
print(rp)

import regex as re

#regex找到[]中的内容
pattern=re.compile(r'\[(.*?)\]')
#找到所有的内容
result=pattern.findall(str(rp))
print(result)
#把所有的内容转换成float元组
result=[tuple(map(float,i.split(','))) for i in result]
print(result)
precision0 = result[0]
recall0 = result[1]
precision1 = result[2]
recall1 = result[3]

# # The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.70 : [0.33,1.00] 0.36 : [0.09,0.62]

          accuracy                                        0.98 : [0.94,1.00]



ModuleNotFoundError: No module named 'regex'

In [4]:

# Part 4
# 4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
#     * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
#     * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
#     * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
#     * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

# complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 
# 
# What would be the cost of having a model that always classifies someone as not calcified on the test set?
# threshold=0.9?


def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]#model.predict_proba(X)计算出来的是一个二维数组，第一列是0的概率，第二列是1的概率，[:,1]取第二列
    predictions = (pred_proba >= threshold)*1#计算出来的是一个bool值，转换成0，1
    #     * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
#     * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**


    false_negatives = (Y == 1) & (predictions == 0)
    false_positives = (Y == 0) & (predictions == 1)
    cost = (30 * false_negatives.sum() + 5 * false_positives.sum()) / len(Y)
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    return cost.mean()

In [5]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

naive_cost = cost(model,1,X_test,Y_test)
print(naive_cost)

0.6976744186046512


In [6]:

# Part 5
# 5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
thresholds = np.linspace(0, 1, 100)
costs=[cost(model,t,X_test,Y_test) for t in thresholds]
cost_at_optimal_threshold = min(costs)
optimal_threshold = thresholds[np.argmin(costs)]

print(optimal_threshold,cost_at_optimal_threshold)

0.11111111111111112 0.3518187239117472


In [7]:

# Part 6
import math
cost_at_optimal_threshold_validation = cost(model,optimal_threshold,X_valid,Y_valid)
# Report the cost interval as a tuple cost_interval = (a,b)
cost_interval_e = math.sqrt(1/(2*len(Y_valid))*math.log(2/0.05))
cost_interval=(max(0,cost_at_optimal_threshold_validation-cost_interval_e),cost_at_optimal_threshold_validation+cost_interval_e)
print(cost_interval)
print(cost_at_optimal_threshold_validation)
# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

(0.26570415858431456, 0.3170630920413558)
0.2913836253128352


In [8]:

# Part 7


C = 30 * (1 - (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)) * Y_valid + 5 * (1 - Y_valid) * (model.predict_proba(X_valid)[:, 1] >= optimal_threshold)
variance_of_C = np.var(C)
variance_of_C


6.198613637958541

In [9]:

# Part 7

# A 95% confidence interval of the random variable C using Bennett's inequality
#实现Bennett不等式
from Utils import bennett_epsilon


n = len(Y_test)
b=30*n
alpha=0.05
sigma = variance_of_C
eps=bennett_epsilon(n,b,sigma,alpha)
interval_of_C=(max(0,variance_of_C-eps),variance_of_C+eps)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)
interval_of_C

Numerical error 1.0408340855860843e-17


(0, 18.89120961417028)

Numerical error -1.1102230246251565e-16
Numerical error -1.1102230246251565e-16
Beginning tests for problem 1

---------------------------------
Beginning test for part1
---------------------------------

-----Beginning test------
Shape of X_train correct it is (6709,6)
-----Ending test---------

-----Beginning test------
Shape of X_test correct it is (1677,6)
-----Ending test---------

-----Beginning test------
Shape of X_valid correct it is (2797,6)
-----Ending test---------

-----Beginning test------
Shape of Y_train correct it is (6709,)
-----Ending test---------

-----Beginning test------
Shape of Y_test correct it is (1677,)
-----Ending test---------

-----Beginning test------
Shape of Y_valid correct it is (2797,)
-----Ending test---------

---------------------------------
Beginning test for part2
---------------------------------

-----Beginning test------
Your model does produce the correct output shape when predict_proba is called on X_train
-----Ending test---------

-----B